# Task 2: Filtering before resizing

Implement a **class `Image3D`**, which should handle 3D medical images with different voxel dimensions,
image sizes and data types. 

### A class constructor __init__ function, which takes a NumPy array representing a 3D image and a tuple of three numerical items for voxel dimension. 

In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import measure
from scipy.interpolate import interpn
from scipy.ndimage import gaussian_filter

A local image coordinate system can be defined as a coordinate system with its origin at the center of the image and its axes aligned with the voxel dimensions. This allows us to represent the position of any voxel in the image using its coordinates in this local coordinate system.

To implement this, we can add a new instance variable voxel_coords to the Image3D class, which will be a NumPy array of shape (nx, ny, nz, 3), where nx, ny, and nz are the dimensions of the image and the last dimension represents the x, y, and z coordinates of each voxel in the local coordinate system. We can pre-compute this array in the constructor function by using the np.meshgrid function and scaling the resulting coordinates by the voxel dimensions.

`meshgrid`: The one with the least elements defines the rows while the other determines the columns.

Another view:
A local image coordinate system can be defined as a coordinate system that is centered at the origin of the image and has its axes aligned with the axes of the image. The voxel coordinates in this coordinate system correspond to the indices of the voxels in the image data array.

In [2]:
# Create a class Image3D
class Image3D:

    # Initialise the class
    def __init__(self, image, voxel_dimension = ()):
        self.image = image
        self.voxel_dimension = voxel_dimension

    # Define a local image coordinate system
        x_coords, y_coords, z_coords = np.mgrid[:self.image.shape[0], :self.image.shape[1], :self.image.shape[2]]
        # Create a 3D array of coordinates
        # The voxel coordinates in this coordinate system correspond to the indices of the voxels in the image data array.
        self.voxel_coords = np.stack([x_coords, y_coords, z_coords], axis = -1)

        ## The voxel_coords variable is a 3D array of shape (self.image.shape[0], self.image.shape[1], self.image.shape[2], 3)
        #  that stores the voxel coordinates in the local image coordinate system.
        # The first three dimensions correspond to the indices of the voxels in the image data array, and the last dimension stores the x, y, and z coordinates of each voxel.
        #self.coords = np.vstack((x_coords.flatten(), y_coords.flatten(), z_coords.flatten())).T

        ## he voxel coordinates are stored as integers,
        # so if you want to convert them to physical coordinates (i.e., coordinates in millimeters or some other unit),
        # you will need to multiply them by the voxel size.
        self.physical_coords = self.voxel_coords * self.voxel_dimension

    # Implement volume_resize which resizes the image data array to a new size
    def volume_resize(self, resize_ratio):

        # Compute the new shape of the resized image by multiplying the original shape by the ratio
        new_shape  = np.round(np.array(self.image.shape) * resize_ratio).astype(int)
        new_coords = np.stack(np.mgrid[:new_shape[0], :new_shape[1], :new_shape[2]], axis = -1)
        #new_coords = new_coords / resize_ratio
        
        # Compute the new voxel dimensions by multiplying the original dimensions by the ratio
        new_voxel_dimension = tuple(np.array(self.voxel_dimesnion) * resize_ratio)

        # Interpolate the filtered data array at the new coordinates
        new_data = interpn((np.arange(self.image.shape[0]), np.arange(self.image.shape[1]), np.arange(self.image.shape[2])), self.image, new_coords, method = 'linear', bounds_error = False, fill_value = 0)
        #new_data = interpn((np.arange(s) for s in self.image.shape), self.image, new_shape, method = 'linear', bounds_error = False, fill_values = 0)

        return Image3D(new_data, new_voxel_dimension)

    # Implement volume_resize_antialias which applies a Gaussian filter to the image data array before resizing it to a new size
    def volume_resize_antialias(self, resize_ratio, sigma):
        # If we filter, do image dimensions change?
        
        # Apply a Gaussian filter to the original image
        filtered_image = gaussian_filter(self.image, sigma = sigma / np.array(self.voxel_dimension))

        # Compute the new shape of the resized image by multiplying the filtered shape by the ratio
        new_shape  = np.round(np.array(filtered_image.shape) * resize_ratio).astype(int)
        new_coords = np.stack(np.mgrid[:new_shape[0], :new_shape[1], :new_shape[2]], axis = -1)
        #new_coords = new_coords / resize_ratio

        # Compute the new voxel dimensions by multiplying the original dimensions by the ratio
        new_voxel_dimension = tuple(np.array(self.voxel_dimesnion) * resize_ratio)

        # Interpolate the filtered data array at the new coordinates
        new_data = interpn((np.arange(filtered_image.shape[0]), np.arange(filtered_image.shape[1]), np.arange(filtered_image.shape[2])), filtered_image, new_coords, method = 'linear', bounds_error = False, fill_value = 0)
        #new_data = interpn((np.arange(s) for s in self.image.shape), self.image, new_shape, method = 'linear', bounds_error = False, fill_values = 0)

        return Image3D(new_data, new_voxel_dimension)


        



In [12]:
a = (1.3,2.5,3.1)
b = (4,5,6)

list(zip(a,b))

int((a))

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'tuple'

In [8]:
6/np.array((2,3,1))

array([3., 2., 6.])

In [6]:
int(2.5)

2

In [28]:
list((np.arange(s) for s in data.shape))

[array([0, 1, 2]), array([0, 1, 2, 3]), array([0, 1, 2, 3, 4])]

In [1]:
import numpy as np
#data = np.random.random((3, 4, 5))
data = np.random.randint(5, size = (3, 4, 5))
x_coords, y_coords, z_coords = np.mgrid[:data.shape[0], :data.shape[1], :data.shape[2]]
np.vstack((x_coords.flatten(), y_coords.flatten(), z_coords.flatten()))
xyz=np.vstack((x_coords.flatten(), y_coords.flatten(), z_coords.flatten())).T
tt=np.stack([x_coords, y_coords, z_coords], axis = -1)

In [6]:
xyz

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 2],
       [0, 0, 3],
       [0, 0, 4],
       [0, 1, 0],
       [0, 1, 1],
       [0, 1, 2],
       [0, 1, 3],
       [0, 1, 4],
       [0, 2, 0],
       [0, 2, 1],
       [0, 2, 2],
       [0, 2, 3],
       [0, 2, 4],
       [0, 3, 0],
       [0, 3, 1],
       [0, 3, 2],
       [0, 3, 3],
       [0, 3, 4],
       [1, 0, 0],
       [1, 0, 1],
       [1, 0, 2],
       [1, 0, 3],
       [1, 0, 4],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 2],
       [1, 1, 3],
       [1, 1, 4],
       [1, 2, 0],
       [1, 2, 1],
       [1, 2, 2],
       [1, 2, 3],
       [1, 2, 4],
       [1, 3, 0],
       [1, 3, 1],
       [1, 3, 2],
       [1, 3, 3],
       [1, 3, 4],
       [2, 0, 0],
       [2, 0, 1],
       [2, 0, 2],
       [2, 0, 3],
       [2, 0, 4],
       [2, 1, 0],
       [2, 1, 1],
       [2, 1, 2],
       [2, 1, 3],
       [2, 1, 4],
       [2, 2, 0],
       [2, 2, 1],
       [2, 2, 2],
       [2, 2, 3],
       [2, 2, 4],
       [2,

In [4]:
tt

array([[[[0, 0, 0],
         [0, 0, 1],
         [0, 0, 2],
         [0, 0, 3],
         [0, 0, 4]],

        [[0, 1, 0],
         [0, 1, 1],
         [0, 1, 2],
         [0, 1, 3],
         [0, 1, 4]],

        [[0, 2, 0],
         [0, 2, 1],
         [0, 2, 2],
         [0, 2, 3],
         [0, 2, 4]],

        [[0, 3, 0],
         [0, 3, 1],
         [0, 3, 2],
         [0, 3, 3],
         [0, 3, 4]]],


       [[[1, 0, 0],
         [1, 0, 1],
         [1, 0, 2],
         [1, 0, 3],
         [1, 0, 4]],

        [[1, 1, 0],
         [1, 1, 1],
         [1, 1, 2],
         [1, 1, 3],
         [1, 1, 4]],

        [[1, 2, 0],
         [1, 2, 1],
         [1, 2, 2],
         [1, 2, 3],
         [1, 2, 4]],

        [[1, 3, 0],
         [1, 3, 1],
         [1, 3, 2],
         [1, 3, 3],
         [1, 3, 4]]],


       [[[2, 0, 0],
         [2, 0, 1],
         [2, 0, 2],
         [2, 0, 3],
         [2, 0, 4]],

        [[2, 1, 0],
         [2, 1, 1],
         [2, 1, 2],
         [2, 1, 3]

In [4]:
np.arange(data.shape[0])

array([0, 1, 2])

In [28]:
ratio = (0.5, 0.5, 0.5);
new_shape = tuple(int(s * r) for s, r in zip(data.shape, ratio))
new_shape

(1, 2, 2)

In [13]:
data.shape

(3, 4, 5)

In [29]:
new_coords = np.stack(np.mgrid[:new_shape[0], :new_shape[1], :new_shape[2]], axis = -1)
new_coords

array([[[[0, 0, 0],
         [0, 0, 1]],

        [[0, 1, 0],
         [0, 1, 1]]]])

In [30]:
new_coords.shape

(1, 2, 2, 3)

In [31]:
new_coords = new_coords / ratio
new_coords

array([[[[0., 0., 0.],
         [0., 0., 2.]],

        [[0., 2., 0.],
         [0., 2., 2.]]]])

In [7]:
from scipy.interpolate import interpn
def value_func_3d(x, y, z):
    return 2 * x + 3 * y - z
x = np.linspace(0, 4, 5)
y = np.linspace(0, 5, 6)
z = np.linspace(0, 6, 7)
points = (x, y, z)
values = value_func_3d(*np.meshgrid(*points, indexing='ij'))

In [10]:
np.meshgrid(*points, indexing='ij')

[array([[[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]],
 
        [[1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.]],
 
        [[2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2.]],
 
        [[3., 3., 3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3., 3., 3.],
         [3., 3., 3., 3., 3., 3., 3.]],
 
        [[4., 4., 4., 4., 4., 4., 4.],
         [4.,

In [9]:
values

array([[[ 0., -1., -2., -3., -4., -5., -6.],
        [ 3.,  2.,  1.,  0., -1., -2., -3.],
        [ 6.,  5.,  4.,  3.,  2.,  1.,  0.],
        [ 9.,  8.,  7.,  6.,  5.,  4.,  3.],
        [12., 11., 10.,  9.,  8.,  7.,  6.],
        [15., 14., 13., 12., 11., 10.,  9.]],

       [[ 2.,  1.,  0., -1., -2., -3., -4.],
        [ 5.,  4.,  3.,  2.,  1.,  0., -1.],
        [ 8.,  7.,  6.,  5.,  4.,  3.,  2.],
        [11., 10.,  9.,  8.,  7.,  6.,  5.],
        [14., 13., 12., 11., 10.,  9.,  8.],
        [17., 16., 15., 14., 13., 12., 11.]],

       [[ 4.,  3.,  2.,  1.,  0., -1., -2.],
        [ 7.,  6.,  5.,  4.,  3.,  2.,  1.],
        [10.,  9.,  8.,  7.,  6.,  5.,  4.],
        [13., 12., 11., 10.,  9.,  8.,  7.],
        [16., 15., 14., 13., 12., 11., 10.],
        [19., 18., 17., 16., 15., 14., 13.]],

       [[ 6.,  5.,  4.,  3.,  2.,  1.,  0.],
        [ 9.,  8.,  7.,  6.,  5.,  4.,  3.],
        [12., 11., 10.,  9.,  8.,  7.,  6.],
        [15., 14., 13., 12., 11., 10.,  9.],
    

In [63]:
point = np.array([2.21, 3.12, 1.15]);point

array([2.21, 3.12, 1.15])

In [ ]:
point = np.array([2.21, 3.12, 1.15])
print(interpn(points, values, point))
[12.63]

In [49]:
new_coords

array([[[[0. , 0. , 0. ],
         [0. , 0. , 1. ],
         [0. , 0. , 2. ],
         [0. , 0. , 3. ],
         [0. , 0. , 4. ]],

        [[0. , 2. , 0. ],
         [0. , 2. , 1. ],
         [0. , 2. , 2. ],
         [0. , 2. , 3. ],
         [0. , 2. , 4. ]]],


       [[[0.5, 0. , 0. ],
         [0.5, 0. , 1. ],
         [0.5, 0. , 2. ],
         [0.5, 0. , 3. ],
         [0.5, 0. , 4. ]],

        [[0.5, 2. , 0. ],
         [0.5, 2. , 1. ],
         [0.5, 2. , 2. ],
         [0.5, 2. , 3. ],
         [0.5, 2. , 4. ]]],


       [[[1. , 0. , 0. ],
         [1. , 0. , 1. ],
         [1. , 0. , 2. ],
         [1. , 0. , 3. ],
         [1. , 0. , 4. ]],

        [[1. , 2. , 0. ],
         [1. , 2. , 1. ],
         [1. , 2. , 2. ],
         [1. , 2. , 3. ],
         [1. , 2. , 4. ]]],


       [[[1.5, 0. , 0. ],
         [1.5, 0. , 1. ],
         [1.5, 0. , 2. ],
         [1.5, 0. , 3. ],
         [1.5, 0. , 4. ]],

        [[1.5, 2. , 0. ],
         [1.5, 2. , 1. ],
         [1.5, 2. 

In [53]:
ratio = (2.0, 0.5, 1.0);
new_shape = tuple(int(s * r) for s, r in zip(data.shape, ratio))
new_coords = np.stack(np.mgrid[:new_shape[0], :new_shape[1], :new_shape[2]], axis = -1) / ratio
new_data = interpn((np.arange(data.shape[0]), np.arange(data.shape[1]), np.arange(data.shape[2])), data, new_coords, method = 'linear', bounds_error = False, fill_value = 0)
data

array([[[1, 2, 0, 2, 0],
        [2, 2, 3, 1, 1],
        [0, 3, 2, 1, 2],
        [0, 0, 1, 0, 0]],

       [[2, 1, 2, 0, 1],
        [4, 2, 4, 2, 0],
        [2, 0, 1, 3, 1],
        [3, 1, 4, 2, 0]],

       [[0, 2, 4, 0, 3],
        [1, 4, 4, 2, 3],
        [4, 1, 3, 4, 0],
        [2, 2, 4, 1, 4]]])

In [50]:
new_data

array([[[1. , 2. , 0. , 2. , 0. ],
        [0. , 3. , 2. , 1. , 2. ]],

       [[1.5, 1.5, 1. , 1. , 0.5],
        [1. , 1.5, 1.5, 2. , 1.5]],

       [[2. , 1. , 2. , 0. , 1. ],
        [2. , 0. , 1. , 3. , 1. ]],

       [[1. , 1.5, 3. , 0. , 2. ],
        [3. , 0.5, 2. , 3.5, 0.5]],

       [[0. , 2. , 4. , 0. , 3. ],
        [4. , 1. , 3. , 4. , 0. ]],

       [[0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. ]]])

In [43]:
new_data

array([[[1. , 2. , 0. , 2. , 0. ],
        [0. , 3. , 2. , 1. , 2. ]],

       [[1.5, 1.5, 1. , 1. , 0.5],
        [1. , 1.5, 1.5, 2. , 1.5]],

       [[2. , 1. , 2. , 0. , 1. ],
        [2. , 0. , 1. , 3. , 1. ]],

       [[1. , 1.5, 3. , 0. , 2. ],
        [3. , 0.5, 2. , 3.5, 0.5]],

       [[0. , 2. , 4. , 0. , 3. ],
        [4. , 1. , 3. , 4. , 0. ]],

       [[0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. ]]])

In [10]:
np.stack(np.mgrid[:new_shape[0], :new_shape[1], :new_shape[2]], axis = -1).shape

(6, 2, 5, 3)

In [11]:
new_shape

(6, 2, 5)

In [12]:
data

array([[[3, 0, 0, 3, 2],
        [2, 2, 0, 1, 1],
        [4, 4, 2, 2, 1],
        [1, 2, 0, 3, 0]],

       [[1, 0, 0, 0, 4],
        [3, 1, 2, 4, 3],
        [1, 4, 3, 2, 2],
        [4, 4, 4, 4, 1]],

       [[2, 1, 3, 2, 1],
        [3, 4, 4, 0, 4],
        [3, 1, 4, 2, 1],
        [3, 1, 0, 4, 3]]])

In [75]:
new_data

array([[[2., 0., 2., 3., 2.],
        [2., 3., 1., 0., 4.]],

       [[1., 3., 1., 1., 2.],
        [2., 1., 4., 0., 4.]],

       [[2., 2., 3., 0., 3.],
        [4., 2., 1., 1., 2.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

In [74]:
new_shape

(6, 2, 5)

In [69]:
np.stack(np.mgrid[:new_shape[0], :new_shape[1], :new_shape[2]], axis = -1)

(6, 2, 5, 3)

In [59]:
data

array([[[2, 0, 2, 3, 2],
        [2, 3, 1, 0, 4],
        [0, 1, 4, 2, 0],
        [4, 4, 2, 2, 3]],

       [[1, 3, 1, 1, 2],
        [2, 1, 4, 0, 4],
        [3, 2, 0, 2, 4],
        [2, 2, 2, 4, 3]],

       [[2, 2, 3, 0, 3],
        [4, 2, 1, 1, 2],
        [4, 3, 1, 4, 0],
        [3, 0, 3, 0, 1]]])

In [58]:
(np.arange(data.shape[0]), np.arange(data.shape[1]), np.arange(data.shape[2]))

(array([0, 1, 2]), array([0, 1, 2, 3]), array([0, 1, 2, 3, 4]))

In [48]:
values

array([[[ 0., -1., -2., -3., -4., -5., -6.],
        [ 3.,  2.,  1.,  0., -1., -2., -3.],
        [ 6.,  5.,  4.,  3.,  2.,  1.,  0.],
        [ 9.,  8.,  7.,  6.,  5.,  4.,  3.],
        [12., 11., 10.,  9.,  8.,  7.,  6.],
        [15., 14., 13., 12., 11., 10.,  9.]],

       [[ 2.,  1.,  0., -1., -2., -3., -4.],
        [ 5.,  4.,  3.,  2.,  1.,  0., -1.],
        [ 8.,  7.,  6.,  5.,  4.,  3.,  2.],
        [11., 10.,  9.,  8.,  7.,  6.,  5.],
        [14., 13., 12., 11., 10.,  9.,  8.],
        [17., 16., 15., 14., 13., 12., 11.]],

       [[ 4.,  3.,  2.,  1.,  0., -1., -2.],
        [ 7.,  6.,  5.,  4.,  3.,  2.,  1.],
        [10.,  9.,  8.,  7.,  6.,  5.,  4.],
        [13., 12., 11., 10.,  9.,  8.,  7.],
        [16., 15., 14., 13., 12., 11., 10.],
        [19., 18., 17., 16., 15., 14., 13.]],

       [[ 6.,  5.,  4.,  3.,  2.,  1.,  0.],
        [ 9.,  8.,  7.,  6.,  5.,  4.,  3.],
        [12., 11., 10.,  9.,  8.,  7.,  6.],
        [15., 14., 13., 12., 11., 10.,  9.],
    

In [43]:
x_*2

array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2.]],

       [[4., 4., 4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4., 4., 4.]],

       [[6., 6., 6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6., 6., 6.]],

       [[8., 8., 8., 8., 8., 8., 8.],
        [8., 8., 8., 8., 8., 8., 8.],
    

In [45]:
y_*3

array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 6.,  6.,  6.,  6.,  6.,  6.,  6.],
        [ 9.,  9.,  9.,  9.,  9.,  9.,  9.],
        [12., 12., 12., 12., 12., 12., 12.],
        [15., 15., 15., 15., 15., 15., 15.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 6.,  6.,  6.,  6.,  6.,  6.,  6.],
        [ 9.,  9.,  9.,  9.,  9.,  9.,  9.],
        [12., 12., 12., 12., 12., 12., 12.],
        [15., 15., 15., 15., 15., 15., 15.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 6.,  6.,  6.,  6.,  6.,  6.,  6.],
        [ 9.,  9.,  9.,  9.,  9.,  9.,  9.],
        [12., 12., 12., 12., 12., 12., 12.],
        [15., 15., 15., 15., 15., 15., 15.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 6.,  6.,  6.,  6.,  6.,  6.,  6.],
        [ 9.,  9.,  9.,  9.,  9.,  9.,  9.],
    

In [47]:
z_

array([[[0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.]],

       [[0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.]],

       [[0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.]],

       [[0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.]],

       [[0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
    

In [40]:
values

array([[[ 0., -1., -2., -3., -4., -5., -6.],
        [ 3.,  2.,  1.,  0., -1., -2., -3.],
        [ 6.,  5.,  4.,  3.,  2.,  1.,  0.],
        [ 9.,  8.,  7.,  6.,  5.,  4.,  3.],
        [12., 11., 10.,  9.,  8.,  7.,  6.],
        [15., 14., 13., 12., 11., 10.,  9.]],

       [[ 2.,  1.,  0., -1., -2., -3., -4.],
        [ 5.,  4.,  3.,  2.,  1.,  0., -1.],
        [ 8.,  7.,  6.,  5.,  4.,  3.,  2.],
        [11., 10.,  9.,  8.,  7.,  6.,  5.],
        [14., 13., 12., 11., 10.,  9.,  8.],
        [17., 16., 15., 14., 13., 12., 11.]],

       [[ 4.,  3.,  2.,  1.,  0., -1., -2.],
        [ 7.,  6.,  5.,  4.,  3.,  2.,  1.],
        [10.,  9.,  8.,  7.,  6.,  5.,  4.],
        [13., 12., 11., 10.,  9.,  8.,  7.],
        [16., 15., 14., 13., 12., 11., 10.],
        [19., 18., 17., 16., 15., 14., 13.]],

       [[ 6.,  5.,  4.,  3.,  2.,  1.,  0.],
        [ 9.,  8.,  7.,  6.,  5.,  4.,  3.],
        [12., 11., 10.,  9.,  8.,  7.,  6.],
        [15., 14., 13., 12., 11., 10.,  9.],
    

In [35]:
x_, y_, z_ =np.meshgrid(*points, indexing='ij')

In [38]:
z_

array([[[0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.]],

       [[0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.]],

       [[0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.]],

       [[0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.]],

       [[0., 1., 2., 3., 4., 5., 6.],
        [0., 1., 2., 3., 4., 5., 6.],
    

In [4]:
# Volume resize
ratio = (2.0, 0.5, 1.0);
new_shape = tuple(int(s * r) for s, r in zip(data.shape, ratio))
new_data = interpn((np.arange(self.image.shape[0]), np.arange(self.image.shape[1]), np.arange(self.image.shape[2])), self.image, np.mgrid[:new_shape[0], :new_shape[1], :new_shape[2]], method = 'linear', bounds_error = False, fill_value = 0)

NameError: name 'self' is not defined

In [21]:
list(np.arange(s) for s in data.shape)

[array([0, 1, 2]), array([0, 1, 2, 3]), array([0, 1, 2, 3, 4])]

In [19]:
np.round(np.array(data.shape) * (0.5, 0.5, 0.5)).astype(int)

array([2, 2, 2])

In [24]:
tuple(np.array((2.0, 4.0, 5.0))*(0.5, 0.5, 0.5))

(1.0, 2.0, 2.5)

In [61]:
y_coords.flatten()

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3])

In [62]:
z_coords.flatten()

array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1,
       2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
       4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4])

In [66]:
np.vstack((x_coords.flatten(), y_coords.flatten(), z_coords.flatten()))

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3],
       [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1,
        2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
        4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4]])

In [67]:
np.vstack((x_coords.flatten(), y_coords.flatten(), z_coords.flatten())).T

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 2],
       [0, 0, 3],
       [0, 0, 4],
       [0, 1, 0],
       [0, 1, 1],
       [0, 1, 2],
       [0, 1, 3],
       [0, 1, 4],
       [0, 2, 0],
       [0, 2, 1],
       [0, 2, 2],
       [0, 2, 3],
       [0, 2, 4],
       [0, 3, 0],
       [0, 3, 1],
       [0, 3, 2],
       [0, 3, 3],
       [0, 3, 4],
       [1, 0, 0],
       [1, 0, 1],
       [1, 0, 2],
       [1, 0, 3],
       [1, 0, 4],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 2],
       [1, 1, 3],
       [1, 1, 4],
       [1, 2, 0],
       [1, 2, 1],
       [1, 2, 2],
       [1, 2, 3],
       [1, 2, 4],
       [1, 3, 0],
       [1, 3, 1],
       [1, 3, 2],
       [1, 3, 3],
       [1, 3, 4],
       [2, 0, 0],
       [2, 0, 1],
       [2, 0, 2],
       [2, 0, 3],
       [2, 0, 4],
       [2, 1, 0],
       [2, 1, 1],
       [2, 1, 2],
       [2, 1, 3],
       [2, 1, 4],
       [2, 2, 0],
       [2, 2, 1],
       [2, 2, 2],
       [2, 2, 3],
       [2, 2, 4],
       [2,

In [63]:
x_coords

array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])

In [58]:
y_coords

array([[[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3]],

       [[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3]],

       [[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3]]])

In [59]:
z_coords

array([[[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]]])

In [47]:
data

array([[[0.98965987, 0.47365407, 0.28457634, 0.96675875, 0.85659439],
        [0.01215394, 0.20252762, 0.74000258, 0.08672393, 0.31377506],
        [0.20548327, 0.03478545, 0.3290931 , 0.4704827 , 0.36741789],
        [0.5094285 , 0.56383063, 0.67557821, 0.5484748 , 0.27110253]],

       [[0.70448991, 0.15729434, 0.89467307, 0.49656436, 0.03836794],
        [0.69198483, 0.94885732, 0.42246418, 0.61870417, 0.23510501],
        [0.98692326, 0.06026998, 0.5284153 , 0.6373397 , 0.99067839],
        [0.73039486, 0.05001397, 0.44278527, 0.86434345, 0.51368395]],

       [[0.70465176, 0.32510373, 0.89259158, 0.95863115, 0.19750949],
        [0.00336991, 0.91987374, 0.03925686, 0.07623683, 0.82927897],
        [0.52356726, 0.78120665, 0.38506039, 0.88402456, 0.69711836],
        [0.44141692, 0.74911826, 0.46598693, 0.52156332, 0.56621526]]])

In [77]:
np.vstack((x_coords.flatten(), y_coords.flatten(), z_coords.flatten())).T.shape

(60, 3)

In [37]:
xx.flatten(); yy.flatten(); zz.flatten()

array([ 0,  1,  2, ..., 27, 28, 29])

In [38]:
yy.flatten()

array([ 0,  0,  0, ..., 19, 19, 19])

In [3]:
import torch

data = torch.randn((100, 200, 300))

In [4]:
data = data.numpy()

In [5]:
data.shape

(100, 200, 300)

In [ ]:
# Pre-compute voxel coordinates in local image coordinate system
x, y, z = np.meshgrid(np.arange(0, data.shape[0]), np.arange(0, data.shape[1]), np.arange(0, data.shape[2]), indexing='ij')

# Create an Image3D object


In [77]:
 
x_coords, y_coords, z_coords = np.meshgrid(
            np.arange(data.shape[0]), np.arange(data.shape[1]), np.arange(data.shape[2]), indexing='ij'
        )
voxel_dimension = (1.0, 1.0, 1.0)

x_coords = x_coords * voxel_dimension[0] - (data.shape[0] - 1) * voxel_dimension[0] / 2.0
y_coords = y_coords * voxel_dimension[1] - (data.shape[1] - 1) * voxel_dimension[1] / 2.0
z_coords = z_coords * voxel_dimension[2] - (data.shape[2] - 1) * voxel_dimension[2] / 2.0

In [78]:
np.stack((x_coords, y_coords, z_coords), axis=-1).shape

(3, 4, 5, 3)

In [98]:
voxel_coords.shape

(100, 200, 300, 3)

In [92]:
voxel_coords

array([[[[ -49.5,  -99.5, -149.5],
         [ -49.5,  -99.5, -148.5],
         [ -49.5,  -99.5, -147.5],
         ...,
         [ -49.5,  -99.5,  147.5],
         [ -49.5,  -99.5,  148.5],
         [ -49.5,  -99.5,  149.5]],

        [[ -49.5,  -98.5, -149.5],
         [ -49.5,  -98.5, -148.5],
         [ -49.5,  -98.5, -147.5],
         ...,
         [ -49.5,  -98.5,  147.5],
         [ -49.5,  -98.5,  148.5],
         [ -49.5,  -98.5,  149.5]],

        [[ -49.5,  -97.5, -149.5],
         [ -49.5,  -97.5, -148.5],
         [ -49.5,  -97.5, -147.5],
         ...,
         [ -49.5,  -97.5,  147.5],
         [ -49.5,  -97.5,  148.5],
         [ -49.5,  -97.5,  149.5]],

        ...,

        [[ -49.5,   97.5, -149.5],
         [ -49.5,   97.5, -148.5],
         [ -49.5,   97.5, -147.5],
         ...,
         [ -49.5,   97.5,  147.5],
         [ -49.5,   97.5,  148.5],
         [ -49.5,   97.5,  149.5]],

        [[ -49.5,   98.5, -149.5],
         [ -49.5,   98.5, -148.5],
         [ -

In [76]:
(data.shape[2] - 1) * voxel_dimension[2] / 2.0

149.5

In [73]:

x_coords = x_coords * voxel_dimension[0] - (data.shape[0] - 1) * voxel_dimension[0] / 2.0

In [71]:
(data.shape[0] - 1) * voxel_dimension[0] / 2.0

49.5

In [79]:
z_coords

array([[[-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        [-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        [-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        ...,
        [-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        [-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        [-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5]],

       [[-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        [-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        [-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        ...,
        [-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        [-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        [-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5]],

       [[-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        [-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        [-149.5, -148.5, -147.5, ...,  147.5,  148.5,  149.5],
        ...,
        [-14

In [29]:
nx, ny = (3, 2)
x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)
xv, yv = np.meshgrid(x, y)
xv

array([[0. , 0.5, 1. ],
       [0. , 0.5, 1. ]])

In [32]:
yv

array([[0., 0., 0.],
       [1., 1., 1.]])

In [30]:
x

array([0. , 0.5, 1. ])

In [31]:
y

array([0., 1.])

In [33]:
np.meshgrid(x, y)

[array([[0. , 0.5, 1. ],
        [0. , 0.5, 1. ]]),
 array([[0., 0., 0.],
        [1., 1., 1.]])]

In [61]:
x = np.array([0, 1, 2])
y = np.array([5, 6, 7])
z = np.array([10, 11, 12])

In [62]:
res=np.array(np.meshgrid(x, y,z, indexing='ij'))
res.shape

(3, 3, 3, 3)

In [65]:
res

array([[[[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],

        [[ 1,  1,  1],
         [ 1,  1,  1],
         [ 1,  1,  1]],

        [[ 2,  2,  2],
         [ 2,  2,  2],
         [ 2,  2,  2]]],


       [[[ 5,  5,  5],
         [ 6,  6,  6],
         [ 7,  7,  7]],

        [[ 5,  5,  5],
         [ 6,  6,  6],
         [ 7,  7,  7]],

        [[ 5,  5,  5],
         [ 6,  6,  6],
         [ 7,  7,  7]]],


       [[[10, 11, 12],
         [10, 11, 12],
         [10, 11, 12]],

        [[10, 11, 12],
         [10, 11, 12],
         [10, 11, 12]],

        [[10, 11, 12],
         [10, 11, 12],
         [10, 11, 12]]]])

In [15]:
m=np.array([[[1,2,3],
            [4,5,6],
            [7,8,9]],
 
            [[10,11,12],
            [13,14,15],
            [16,17,18]]])
 
n=np.array([[[51,52,53],
            [54,55,56],
            [57,58,59]],
 
            [[110,111,112],
            [113,114,115],
            [116,117,118]]])

In [16]:
m.shape

(2, 3, 3)

In [17]:
m[1]

array([[10, 11, 12],
       [13, 14, 15],
       [16, 17, 18]])

In [18]:
np.stack((m,n),axis=-1)


array([[[[  1,  51],
         [  2,  52],
         [  3,  53]],

        [[  4,  54],
         [  5,  55],
         [  6,  56]],

        [[  7,  57],
         [  8,  58],
         [  9,  59]]],


       [[[ 10, 110],
         [ 11, 111],
         [ 12, 112]],

        [[ 13, 113],
         [ 14, 114],
         [ 15, 115]],

        [[ 16, 116],
         [ 17, 117],
         [ 18, 118]]]])

In [10]:
x_coords, y_coords, z_coords = np.meshgrid(
            np.arange(data.shape[0]), np.arange(data.shape[1]), np.arange(data.shape[2]), indexing='ij'
        )

voxel_coords2 = np.stack((x_coords, y_coords, z_coords), axis=-1)


In [12]:
voxel_coords2

array([[[[  0,   0,   0],
         [  0,   0,   1],
         [  0,   0,   2],
         ...,
         [  0,   0, 297],
         [  0,   0, 298],
         [  0,   0, 299]],

        [[  0,   1,   0],
         [  0,   1,   1],
         [  0,   1,   2],
         ...,
         [  0,   1, 297],
         [  0,   1, 298],
         [  0,   1, 299]],

        [[  0,   2,   0],
         [  0,   2,   1],
         [  0,   2,   2],
         ...,
         [  0,   2, 297],
         [  0,   2, 298],
         [  0,   2, 299]],

        ...,

        [[  0, 197,   0],
         [  0, 197,   1],
         [  0, 197,   2],
         ...,
         [  0, 197, 297],
         [  0, 197, 298],
         [  0, 197, 299]],

        [[  0, 198,   0],
         [  0, 198,   1],
         [  0, 198,   2],
         ...,
         [  0, 198, 297],
         [  0, 198, 298],
         [  0, 198, 299]],

        [[  0, 199,   0],
         [  0, 199,   1],
         [  0, 199,   2],
         ...,
         [  0, 199, 297],
        

In [13]:
vvoxel_coords

array([[[[  0,   0,   0],
         [  0,   0,   1],
         [  0,   0,   2],
         ...,
         [  0,   0, 297],
         [  0,   0, 298],
         [  0,   0, 299]],

        [[  0,   1,   0],
         [  0,   1,   1],
         [  0,   1,   2],
         ...,
         [  0,   1, 297],
         [  0,   1, 298],
         [  0,   1, 299]],

        [[  0,   2,   0],
         [  0,   2,   1],
         [  0,   2,   2],
         ...,
         [  0,   2, 297],
         [  0,   2, 298],
         [  0,   2, 299]],

        ...,

        [[  0, 197,   0],
         [  0, 197,   1],
         [  0, 197,   2],
         ...,
         [  0, 197, 297],
         [  0, 197, 298],
         [  0, 197, 299]],

        [[  0, 198,   0],
         [  0, 198,   1],
         [  0, 198,   2],
         ...,
         [  0, 198, 297],
         [  0, 198, 298],
         [  0, 198, 299]],

        [[  0, 199,   0],
         [  0, 199,   1],
         [  0, 199,   2],
         ...,
         [  0, 199, 297],
        

In [ ]:
import numpy as np

class Image3D:
    def __init__(self, data: np.ndarray, voxel_dimension: tuple[float, float, float]):
        self.data = data
        self.voxel_dimension = voxel_dimension

        # Pre-compute voxel coordinates in local image coordinate system
        x_coords, y_coords, z_coords = np.meshgrid(
            np.arange(data.shape[0]), np.arange(data.shape[1]), np.arange(data.shape[2]), indexing='ij'
        )
        x_coords = x_coords * voxel_dimension[0] - (data.shape[0] - 1) * voxel_dimension[0] / 2.0
        y_coords = y_coords * voxel_dimension[1] - (data.shape[1] - 1) * voxel_dimension[1] / 2.0
        z_coords = z_coords * voxel_dimension[2] - (data.shape[2] - 1) * voxel_dimension[2] / 2.0
        self.voxel_coords = np.stack((x_coords, y_coords, z_coords), axis=-1)

    @property
    def shape(self) -> tuple[int, int, int]:
        return self.data.shape

    @property
    def voxel_size(self) -> tuple[float, float, float]:
        return self.voxel_dimension
    
    @property
    def dtype(self) -> np.dtype:
        return self.data.dtype


In [24]:
data = np.random.random((10, 20, 30))
xx, yy, zz = np.mgrid[:data.shape[0], :data.shape[1], :data.shape[2]]
vvoxel_coords = np.stack([xx, yy, zz], axis=-1)

In [25]:
vvoxel_coords

array([[[[ 0,  0,  0],
         [ 0,  0,  1],
         [ 0,  0,  2],
         ...,
         [ 0,  0, 27],
         [ 0,  0, 28],
         [ 0,  0, 29]],

        [[ 0,  1,  0],
         [ 0,  1,  1],
         [ 0,  1,  2],
         ...,
         [ 0,  1, 27],
         [ 0,  1, 28],
         [ 0,  1, 29]],

        [[ 0,  2,  0],
         [ 0,  2,  1],
         [ 0,  2,  2],
         ...,
         [ 0,  2, 27],
         [ 0,  2, 28],
         [ 0,  2, 29]],

        ...,

        [[ 0, 17,  0],
         [ 0, 17,  1],
         [ 0, 17,  2],
         ...,
         [ 0, 17, 27],
         [ 0, 17, 28],
         [ 0, 17, 29]],

        [[ 0, 18,  0],
         [ 0, 18,  1],
         [ 0, 18,  2],
         ...,
         [ 0, 18, 27],
         [ 0, 18, 28],
         [ 0, 18, 29]],

        [[ 0, 19,  0],
         [ 0, 19,  1],
         [ 0, 19,  2],
         ...,
         [ 0, 19, 27],
         [ 0, 19, 28],
         [ 0, 19, 29]]],


       [[[ 1,  0,  0],
         [ 1,  0,  1],
         [ 1, 

In [23]:
zz

array([[[ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29],
        ...,
        [ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29]],

       [[ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29],
        ...,
        [ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29]],

       [[ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29],
        ...,
        [ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29]],

       ...,

       [[ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29],
        ...,
        [ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28, 29],
        [ 0,  1,  2, ..., 27, 28

In [14]:
vvoxel_coords

array([[[[  0,   0,   0],
         [  0,   0,   1],
         [  0,   0,   2],
         ...,
         [  0,   0, 297],
         [  0,   0, 298],
         [  0,   0, 299]],

        [[  0,   1,   0],
         [  0,   1,   1],
         [  0,   1,   2],
         ...,
         [  0,   1, 297],
         [  0,   1, 298],
         [  0,   1, 299]],

        [[  0,   2,   0],
         [  0,   2,   1],
         [  0,   2,   2],
         ...,
         [  0,   2, 297],
         [  0,   2, 298],
         [  0,   2, 299]],

        ...,

        [[  0, 197,   0],
         [  0, 197,   1],
         [  0, 197,   2],
         ...,
         [  0, 197, 297],
         [  0, 197, 298],
         [  0, 197, 299]],

        [[  0, 198,   0],
         [  0, 198,   1],
         [  0, 198,   2],
         ...,
         [  0, 198, 297],
         [  0, 198, 298],
         [  0, 198, 299]],

        [[  0, 199,   0],
         [  0, 199,   1],
         [  0, 199,   2],
         ...,
         [  0, 199, 297],
        

This line of code scales the x_coords array by the x voxel dimension and then translates it so that the voxel at the center of the image has an x coordinate of 0.

The x_coords array is an array of integers representing the indices of the voxels along the x axis. By multiplying it by the x voxel dimension, we convert the indices to physical distances in millimeters.

Then, we subtract (data.shape[0] - 1) * voxel_dimension[0] / 2.0 from the resulting array. This value is equal to the distance between the center of the image and the leftmost voxel along the x axis. By subtracting it, we translate the x_coords array so that the voxel at the center of the image has an x coordinate of 0.

The same process is repeated for the y_coords and z_coords arrays to define the local image coordinate system.